In [1]:
import load
import pandas as pd
import numpy as np
import keras
import h5py

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.regularizers import WeightRegularizer, ActivityRegularizer, l2, activity_l2
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.utils import np_utils, generic_utils

import theano
theano.config.compute_test_value = 'off'

Let's load up the metadata.

In [4]:
meta_df = pd.DataFrame.from_csv('/home/ubuntu/metaframe.csv', index_col=False)
meta_df['concentration'] = meta_df.concentration.apply(lambda x: '%0.3f' % x)
meta_df['replicate'] = meta_df.replicate.apply(lambda x: str(x))

We need to define a dictionary from our model targets (the compounds) to integers.

In [5]:
target_mapper = dict(zip(meta_df['compound'].unique(),
                         np.arange(meta_df['compound'].nunique())))

Here, we define a callback necessary for validating on a generator while also training on a generator.

In [6]:
from keras.callbacks import Callback

class FineHistory(Callback):
    def __init__(self, model, val_generator, n_samples=(2 ** 15)):
        self.batch_loss = []
        self.batch_size = []
        self.epoch_loss = []
        self.val_loss = []
        self.val_acc = []
        self.val_roc = []
        self.model = model
        self.val_generator = val_generator
        self.n_samples = n_samples
    
    def on_epoch_begin(self, epoch, logs={}):
        self.batch_loss.append([])
        self.batch_size.append([])
        
    def on_epoch_end(self, epoch, logs={}):
        losses = np.array(self.batch_loss[-1])
        sizes = np.array(self.batch_size[-1])
        self.epoch_loss.append(np.dot(losses, sizes) / np.sum(sizes))
        self.batch_loss[-1] = losses
        self.batch_size[-1] = sizes
        val_loss, val_acc = self.model.evaluate_generator(self.val_generator, self.n_samples, verbose=0, show_accuracy=True)
        self.val_loss.append(val_loss)
        self.val_acc.append(val_acc)
        logs['val_loss'] = self.val_loss[-1]
        logs['val_acc'] = self.val_acc[-1]
        
    def on_batch_end(self, batch, logs={}):
        self.batch_loss[-1].append(logs.get('loss'))
        self.batch_size[-1].append(logs.get('size'))
        
def plot_cb(callback, y_range=None):    
    p_loss = bop.figure(y_range=y_range)
    p_loss.line(np.arange(0, len(callback.epoch_loss)),
                callback.epoch_loss,
                legend='Train Loss',
                color='Blue')
    p_loss.line(np.arange(0, len(callback.val_loss)),
                callback.val_loss,
                legend='Validation Loss',
                color='Green')
    
    bop.show(p_loss)

Here we define a simple keras model that is 3 layers deep. We have 2 convolutional layers (along with activations, pooling, and dropout). We also have a fully-connected layer as well.

In [7]:
from keras.regularizers import WeightRegularizer

def create_model(nb_classes, img_rows, img_cols, img_channels):
    model = Sequential()

    model.add(AveragePooling2D(pool_size=(3, 3), input_shape=(img_channels, img_rows, img_cols)))
    model.add(Convolution2D(32, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    return model

In [8]:
nb_classes = len(target_mapper)
img_rows, img_cols = 192, 192
img_channels = 3

model = create_model(nb_classes, img_rows, img_cols, img_channels)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Let's define our data generators using the `load` module (feel free to modify this). For each record of the passed metadata frame (whose rows are `compound`, `concentration`, `replicate`, and `moa`), a single randomly cropped image of the specified size will be returned in a minibatch.

In [16]:
ds = h5py.File('/home/ubuntu/morphological_profiling_dl.h5', 'r')

train_generator = load.minibatch_generator(ds, meta_df.query("replicate in ('1', '2')"), target_mapper, size=192)
validation_generator = load.minibatch_generator(ds, meta_df.query("replicate == '3'"), target_mapper, size=192)

cb = FineHistory(model, validation_generator, 1000)

In [38]:
nb_epoch = 25
history = model.fit_generator(train_generator, callbacks=[cb],
                              samples_per_epoch=5000,
                              nb_epoch=nb_epoch, show_accuracy=True,
                              nb_worker=5)

Epoch 1/5
10098/10000 [==============================] - 229s - loss: 3.3900 - acc: 0.1065 - val_loss: 3.1685 - val_acc: 0.1350
Epoch 2/5
10098/10000 [==============================] - 258s - loss: 2.9705 - acc: 0.1719 - val_loss: 2.7980 - val_acc: 0.2114
Epoch 3/5
10098/10000 [==============================] - 281s - loss: 2.6244 - acc: 0.2237 - val_loss: 2.5545 - val_acc: 0.2345
Epoch 4/5
10098/10000 [==============================] - 287s - loss: 2.4201 - acc: 0.2628 - val_loss: 2.3733 - val_acc: 0.2664
Epoch 5/5
10098/10000 [==============================] - 292s - loss: 2.2602 - acc: 0.2914 - val_loss: 2.2643 - val_acc: 0.2880


Cool, we now have a trained model. Let's define a second model that will return the activations at the second to last layer. This can be considered a profile.

In [96]:
def create_base_network(layers):
    seq = Sequential()
    seq.add(AveragePooling2D(pool_size=(3, 3), input_shape=(3, 192, 192)))

    seq.add(Convolution2D(32, 3, 3, border_mode='same', weights=layers[1].get_weights()))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(4, 4)))
    seq.add(Dropout(0.25))
    
    seq.add(Convolution2D(64, 3, 3, border_mode='same', weights=layers[5].get_weights()))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(4, 4)))
    seq.add(Dropout(0.25))
    
    seq.add(Flatten())
    seq.add(Dense(256, weights=layers[10].get_weights()))
    
    return seq

def get_activation_function(model):
    layers = model.layers
    seq = create_base_network(layers)
    get_activations = theano.function([seq.layers[0].input], seq.layers[-1].get_output(train=False), allow_input_downcast=True)
    return get_activations

We also define some functions for calculating the distances between a bunch of vectors in a pandas dataframe.

In [52]:
from scipy.spatial.distance import cdist

def df_cdist(df1, df2, *args, **kwargs):
    """
    Returns the matrix of pair-wise distances between the rows of df1 and
    the rows of df2. Also works with Series.
    We use scipy.spatial.distance.cdist (see online docs
    http://docs.scipy.org/doc/scipy/reference/generated/
    scipy.spatial.distance.cdist.html#scipy.spatial.distance.cdist)
        
    Parameters
    ----------
    df1 : pandas DataFrame
        First dataframe.
                                       
    df2 : pandas DataFrame       
        Second dataframe.
    
    args : arguments
        Additional positional arguments.
    
    kwargs : arguments
        Additional key-value arguments.

    Returns
    -------
    distFrame : pandas DataFrame
        DataFrame with N rows (where N = `len(df1)`) and M columns
        (where M = `len(df2)`). The i,j^th entry is the distance
        between row `i` of `df1` and row `j` of `df2`.
    """
    # Handle Series
    if isinstance(df1, pd.Series) and isinstance(df2, pd.Series):
        distFrame = df_cdist(pd.DataFrame(df1).T, pd.DataFrame(df2).T,
                             *args, **kwargs).iloc[0].iloc[0]
    elif isinstance(df1, pd.Series):
        distFrame = df_cdist(pd.DataFrame(df1).T, df2, *args, **kwargs).iloc[0]
    elif isinstance(df2, pd.Series):
        distFrame = df_cdist(df1, pd.DataFrame(df2).T, *args, **kwargs).icol(0)
    else:
        distFrame = pd.DataFrame(cdist(df1.values, df2.values,
                                       *args, **kwargs),
                                 index=df1.index, columns=df2.index)
    return distFrame


def df_cangle(df1, df2):
    # clip because of numerical inaccuracy
    costh = (1-df_cdist(df1, df2, metric='cosine')).clip(-1, 1)
    return np.rad2deg(np.arccos(costh))

In [97]:
activation_function = get_activation_function(model)

Let's load up some data...

In [65]:
Xs = []
ys = []
for _ in range(20):
    X, y = train_generator.next()
    Xs.append(X)
    ys.append(y)
    
X = np.concatenate(Xs)
Y = np.concatenate(ys)

In [72]:
inv_target_mapper = {v: k for k, v in target_mapper.iteritems()}

... and calculate the angles between the profiles.

In [99]:
latent_reps = pd.DataFrame(activation_function(X))
latent_reps['drug'] = map(lambda x: inv_target_mapper[x], np.argmax(Y, axis=1))

angles = df_cangle(latent_reps.groupby(['moa', 'drug']).mean(),
                   latent_reps.groupby(['moa', 'drug']).mean())

In [103]:
angles